In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil

In [3]:
def load_dataset(dataset_path):
    image_paths = []
    labels = []
    
    for label in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, label)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                image_paths.append(img_path)
                labels.append(label)
    
    return pd.DataFrame({'image_path': image_paths, 'label': labels})


In [8]:
# Split dataset
def split_dataset(df, test_size=0.2):
    train_df, test_df = train_test_split(df, test_size=test_size, stratify=df['label'], random_state=42)
    return train_df, test_df

In [9]:
# Data Augmentation
def create_data_generators(train_df, test_df, img_size=(227, 227), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                 height_shift_range=0.2, horizontal_flip=True)
    
    train_generator = datagen.flow_from_dataframe(
        train_df, x_col='image_path', y_col='label', target_size=img_size,
        batch_size=batch_size, class_mode='categorical' if len(train_df['label'].unique()) == 2 else 'categorical'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        test_df, x_col='image_path', y_col='label', target_size=img_size,
        batch_size=batch_size, class_mode='categorical' if len(train_df['label'].unique()) == 2 else 'categorical'
    )

    return train_generator, test_generator

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from preprocess import load_dataset, split_dataset, create_data_generators

,image_path,label
8494,../dataset\Negative\08495.jpg,Negative
29965,../dataset\Positive\09966.jpg,Positive
4302,../dataset\Negative\04303.jpg,Negative
17489,../dataset\Negative\17490.jpg,Negative
29855,../dataset\Positive\09856.jpg,Positive
...,...,...
37723,../dataset\Positive\17724_1.jpg,Positive
37930,../dataset\Positive\17931_1.jpg,Positive
8040,../dataset\Negative\08041.jpg,Negative
36247,../dataset\Positive\16248_1.jpg,Positive


In [ ]:
# Load and preprocess data
dataset_path = "./dataset"
df = load_dataset(dataset_path)
train_df, test_df = split_dataset(df)
train_generator, test_generator = create_data_generators(train_df, test_df)

In [37]:
# Model Architectures
def build_model(base_model_name="ResNet50", input_shape=(227, 227, 3), num_classes=2):
    if base_model_name == "ResNet50":
        base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    elif base_model_name == "MobileNetV2":
        base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Invalid model name")

    base_model.trainable = False
    x = Flatten()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation="sigmoid" if num_classes == 2 else "softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss="binary_crossentropy" if num_classes == 2 else "categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model 


(2, 2)

In [40]:
# Train and save models
resnet_model = build_model("ResNet50", num_classes=len(df['label'].unique()))
mobilenet_model = build_model("MobileNetV2", num_classes=len(df['label'].unique()))

resnet_model.fit(train_generator, validation_data=test_generator, epochs=2)
resnet_model.save("models/resnet_model.h5")

mobilenet_model.fit(train_generator, validation_data=test_generator, epochs=2)
mobilenet_model.save("models/mobilenet_model.h5")


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
#from preprocess import create_data_generators, split_dataset, load_dataset

In [ ]:
df = load_dataset("./dataset")
train_df, test_df = split_dataset(df)
_, test_generator = create_data_generators(train_df, test_df)


In [ ]:
# Load trained models
resnet_model = tf.keras.models.load_model("models/resnet_model.h5")
mobilenet_model = tf.keras.models.load_model("models/mobilenet_model.h5")

In [ ]:
# Evaluate models
def evaluate_model(model, test_generator):
    y_true = test_generator.classes
    y_pred = np.argmax(model.predict(test_generator), axis=1)

    print(f"Model Evaluation:\n{classification_report(y_true, y_pred)}")

evaluate_model(resnet_model, test_generator)
evaluate_model(mobilenet_model, test_generator)
